In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
#from pandas_ml import ConfusionMatrix
from matplotlib import pyplot as plt
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_extraction.text import HashingVectorizer
import itertools
import numpy as np

import io

In [ ]:
 from google.colab import files
  
  
 uploaded = files.upload()

Saving train.tsv to train.tsv


In [ ]:
df_train = pd.read_csv(io.BytesIO(uploaded['train.tsv']), sep='\t',header=None)
 

In [ ]:
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...


In [ ]:
#NLTK Stop words
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# df_test = pd.read_csv("C:/Users/James/Documents/Spring-2021/Mining Massive Datasets/liar_dataset/test.tsv", sep='\t',header=None)
# df_train = pd.read_csv("C:/Users/James/Documents/Spring-2021/Mining Massive Datasets/liar_dataset/train.tsv", sep='\t',header=None)
# df_valid = pd.read_csv("C:/Users/James/Documents/Spring-2021/Mining Massive Datasets/liar_dataset/valid.tsv", sep='\t',header=None)

In [ ]:
test = df_train[(df_train[1] =="half-true" ) | (df_train[1] =="mostly-true" ) | (df_train[1] =="true" ) ]
test1 = df_train[(df_train[1] =="false" ) | (df_train[1] =="barely-true" ) | (df_train[1] =="pants-fire" ) ]

In [ ]:
len(test)

5752

In [ ]:
len(test1)

4488

In [ ]:
test = test.sample(4488)
frames = [test, test1]
df_train_data = pd.concat(frames)

In [ ]:
df_credit = df_train_data[[8,9, 10, 11, 12]]

In [ ]:
df_train = df_train_data[[1,2,4,5,6,7,8,9,10,11,12,13]]

In [ ]:
#df_train.rename(columns={1:"label"},inplace=True)

In [ ]:
df_train_data.rename(columns={0:"file",1:"label",2:"Overview",3:"Headline",4:"Author",5:"Authors_Title",6:"State_Info",7:"Party",8:"Credit_One",9:"Credit_Two",10:"Credit_Three",11:"Credit_Four",12:"Credit_Five",13:"Venue"},inplace=True)
df_train_data[['Credit_One', 'Credit_Two', 'Credit_Three','Credit_Four','Credit_Five']] = df_train_data[['Credit_One', 'Credit_Two', 'Credit_Three','Credit_Four','Credit_Five']].astype(str)

In [ ]:
thisdict = {
  "barely-true": "0",
  "false": "0",
  "half-true" : "1",
  "mostly-true" : "1",
  "pants-fire" : "0",
  "true" : "1"
}

In [ ]:
def real_news_or_fake_news(x):
    for k,v in thisdict.items():
        if x == k:
            x = str(v)
            return x

In [ ]:
df_train_data["label"] = df_train_data.apply( lambda x: real_news_or_fake_news(x["label"]), axis= 1 )

In [ ]:
#NLTK Stop words
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
messages = df_train_data.copy(deep=True)

In [ ]:
messages["Party"].replace("none", "Unknown",inplace=True)
messages["Author"].fillna('Unknown', inplace=True)
messages["Party"].fillna('Unknown', inplace=True)
messages["Authors_Title"].fillna('Unknown', inplace=True)
messages["State_Info"].fillna('Unknown', inplace=True)
messages["Venue"].fillna('Unknown', inplace=True)

In [ ]:
messages['Credit_One'] = messages['Credit_One'].replace(np.nan, 0)
messages['Credit_Two'] = messages['Credit_Two'].replace(np.nan, 0)
messages['Credit_Three'] = messages['Credit_Three'].replace(np.nan, 0)
messages['Credit_Four'] = messages['Credit_Four'].replace(np.nan, 0)
messages['Credit_Five'] = messages['Credit_Five'].replace(np.nan, 0)

In [ ]:
import string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text


In [ ]:
import re
messages["Overview"] = messages["Overview"].apply(wordopt)
messages["Party"] = messages["Party"].apply(wordopt)
messages["Authors_Title"] = messages["Authors_Title"].apply(wordopt)
messages["Author"] = messages["Author"].apply(wordopt)
messages["Venue"] = messages["Venue"].apply(wordopt)
messages["State_Info"] = messages["State_Info"].apply(wordopt)


In [ ]:
df_new = messages['Overview'].astype(str) + messages['Authors_Title'] + " " + messages['Author'] + " " + messages['State_Info'] + " " + messages['Author'] + " " + messages['Venue'] + " " + messages['Credit_One'] + " " + messages['Credit_Two'] + " " + messages['Credit_Three'] + " " + messages['Credit_Four'] + " " + messages['Credit_Five']

In [ ]:
df_new = pd.DataFrame(df_new)

In [ ]:
df_new.rename(columns={0:'text' },inplace=True)

In [ ]:
df_new["text"].iloc[0]

'says king street patriots held a fundraiser featuring an author who believes that registering the poor to vote is un american texas democratic party chair boyd richie texas boyd richie an email blast 2.0 1.0 0.0 0.0 1.0'

In [ ]:
df_new

,text
1533,says king street patriots held a fundraiser fe...
1904,says donald trump said he intends to open up a...
1556,without a medicaid expansion more ohioans c...
6733,the health care bill dumps to million low i...
2865,says four year terms for city council members ...
...,...
10228,stopped by smiley cookie to pick up some great...
10230,the supreme courts views are radically out of ...
10233,mayor fung wants to punish our childrens educa...
10238,on lifting the u s cuban embargo and allowing...


In [ ]:
ps = PorterStemmer()
corpus = []
words = []
for i in range(0,len(df_new)):
    review = re.sub('[^a-zA-Z0-9]',' ',df_new["text"].iloc[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus.append(statements)
    words.append(review)

In [ ]:
def clean_text_data(row):
    # convert text to lower case
    row = row.lower()
    
    # this line of code remove non letters
    #row = re.sub('[^A-Za-z0-9.]', ' ', row)
    row = row.split()
    
    review = [ps.stem(word) for word in row if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    return review

In [ ]:
df_new["text"] = df_new.apply( lambda x: clean_text_data(x["text"]), axis=1)

In [ ]:
df_new["text"].iloc[0]

'say king street patriot held fundrais featur author believ regist poor vote un american texa democrat parti chair boyd richi texa boyd richi email blast 2.0 1.0 0.0 0.0 1.0'

In [ ]:
df_credit.rename(columns={8:"Credit_One", 9:"Credit_Two",10:"Credit_Three",11:"Credit_Four",12:"Credit_Five"},inplace=True)
one_hot = pd.get_dummies(df_credit, columns=['Credit_One',"Credit_Two","Credit_Three","Credit_Four","Credit_Five"])


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#one_hot.rename(columns={0:"file"},inplace=True)
one_hot

,Credit_One_0.0,Credit_One_1.0,Credit_One_2.0,Credit_One_3.0,Credit_One_4.0,Credit_One_5.0,Credit_One_6.0,Credit_One_7.0,Credit_One_8.0,Credit_One_9.0,Credit_One_10.0,Credit_One_11.0,Credit_One_12.0,Credit_One_13.0,Credit_One_14.0,Credit_One_15.0,Credit_One_16.0,Credit_One_17.0,Credit_One_18.0,Credit_One_19.0,Credit_One_26.0,Credit_One_28.0,Credit_One_30.0,Credit_One_31.0,Credit_One_33.0,Credit_One_34.0,Credit_One_36.0,Credit_One_40.0,Credit_One_63.0,Credit_One_70.0,Credit_Two_0.0,Credit_Two_1.0,Credit_Two_2.0,Credit_Two_3.0,Credit_Two_4.0,Credit_Two_5.0,Credit_Two_6.0,Credit_Two_7.0,Credit_Two_8.0,Credit_Two_9.0,...,Credit_Four_7.0,Credit_Four_8.0,Credit_Four_10.0,Credit_Four_11.0,Credit_Four_12.0,Credit_Four_14.0,Credit_Four_15.0,Credit_Four_16.0,Credit_Four_18.0,Credit_Four_19.0,Credit_Four_23.0,Credit_Four_24.0,Credit_Four_33.0,Credit_Four_34.0,Credit_Four_35.0,Credit_Four_37.0,Credit_Four_40.0,Credit_Four_41.0,Credit_Four_76.0,Credit_Four_163.0,Credit_Five_0.0,Credit_Five_1.0,Credit_Five_2.0,Credit_Five_3.0,Credit_Five_4.0,Credit_Five_5.0,Credit_Five_6.0,Credit_Five_7.0,Credit_Five_8.0,Credit_Five_9.0,Credit_Five_10.0,Credit_Five_11.0,Credit_Five_15.0,Credit_Five_16.0,Credit_Five_18.0,Credit_Five_19.0,Credit_Five_36.0,Credit_Five_44.0,Credit_Five_61.0,Credit_Five_105.0
1533,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1904,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1556,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6733,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2865,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10230,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
10233,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10238,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
one_hot.isnull().values.any()

False

In [ ]:
df_new.isnull().values.any()

False

In [ ]:
#df_next = pd.merge(df_new, df_credit, on="file", how='inner')
df_new.head(2)

,text
1126,say unemploy tripl dane counti due kathleen fa...
22,best year auto industri america histori presid...


In [ ]:
# frames1 = [df_new,df_credit]
# df_new = pd.concat(frames1, axis=1)

In [ ]:
df_new.columns

Index(['text'], dtype='object')

In [ ]:
df_credit

,Credit_One,Credit_Two,Credit_Three,Credit_Four,Credit_Five
1533,2.0,1.0,0.0,0.0,1.0
1904,3.0,1.0,8.0,10.0,0.0
1556,9.0,8.0,10.0,18.0,3.0
6733,1.0,2.0,2.0,0.0,0.0
2865,1.0,0.0,4.0,1.0,0.0
...,...,...,...,...,...
10228,63.0,114.0,51.0,37.0,61.0
10230,36.0,33.0,15.0,19.0,8.0
10233,0.0,0.0,0.0,0.0,1.0
10238,3.0,1.0,3.0,0.0,0.0


In [ ]:
y = messages["label"]

In [ ]:
# Make training and test sets 
X_train, X_test, y_train, y_test = train_test_split(one_hot, y, test_size=0.30, random_state=53)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(n_estimators=200, random_state=0)
RFC.fit(X_train , y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
RFC.score(X_test, y_test)

0.6899368733754178

In [ ]:
#Predict the response for test dataset
y_pred = RFC.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[895 424]
 [411 963]]
              precision    recall  f1-score   support

           0       0.69      0.68      0.68      1319
           1       0.69      0.70      0.70      1374

    accuracy                           0.69      2693
   macro avg       0.69      0.69      0.69      2693
weighted avg       0.69      0.69      0.69      2693

